# Systemy uczące się - Zad. dom. 2: Maszyny wektorów podpierających

### Autor rozwiązania: Maciej Wieczorek 148141

Ten notebook zawiera zadania związane z maszynami wektorów podpierających (SVM).
Do notebooka zostały dołączony plik helpers.py oraz dwa pliki .npz z danymi, które są używane w zadaniach, 
nie musisz do nich zaglądać ani ich modyfikować. Notebook jest sprawdzany półautomatycznie - przed wysyłką sprawdź czy cały kod wykonuje się bez błędów.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Zadanie 1. - Implementacja maszyny wektorów podpierających (za 20%)

Zadanie polega na zaimplementowaniu maszyny wektorów podpierających z wykorzystaniem biblioteki [`cvxpy`](https://github.com/cvxpy/cvxpy) zapewniającej solver do programowania kwadratowego. Rozpocznijmy od szybkiej nauki jak ta biblioteka działa:


In [ ]:
!pip install cvxpy

- Optymalizacja bez ograniczeń: znajdź maksimum funkcji $$ - (x - 5 )^2 + 10$$

In [ ]:
from cvxpy import *
x = Variable(1) #zmienna optymalizowana, wektor o długości 1 (bo to jedna liczba)

objective = Maximize(-(x-5)**2 + 10) # Implementacja funkcji celu ze wskazanym kierunkiem maksymalizacji
prob = Problem(objective) # Stworzenie problemu optymalizacji

print(prob.solve()) #Rozwiąż i zwróć wartość funkcji celu
print(x.value) # Znaleziona wartość x
print(prob.status) 

- optymalizacja z ograniczeniami: znajdź minimum funkcji $e^{(x-2)^2}$ dla $x>0$.

In [ ]:
x = Variable(1)

objective = Minimize(exp((x-2)**2))
constraints = [x>=0]
prob = Problem(objective, constraints) # Problem teraz składa się z funkcji celu i *listy* ograniczeń

print(prob.solve())
print(x.value)
print(prob.status) 

- optymalizacja z ograniczeniami: znajdź minimum funkcji $x + y$ dla $x\geq 0, y\geq 6$.

In [ ]:
x = Variable(1)
y = Variable(1)

objective = Minimize(x + y)
constraints = [x>=0, y>=6] # <- To zwykła pythonowa lista
prob = Problem(objective, constraints) # Problem teraz składa się z funkcji celu i listy ograniczeń

print(prob.solve())
print(x.value, y.value)
print(prob.status) 

Alternatywna postać tego samego problemu, wykorzystująca notację wektorową:

In [ ]:
x = Variable(2)

objective = Minimize(sum(x))
constraints = [x[0]>=0, x[1]>=6]
prob = Problem(objective, constraints) # Problem teraz składa się z funkcji celu i listy ograniczeń

print(prob.solve())
print(x.value)
print(prob.status) 

Wygenerujmy i narysujmy próbkę dwuwymiarowych danych, które są separowalne liniowo. Zwróć uwagę, że dla uproszczenia przyszłej implementacji każda z cech jest przechowywana w osobnym wektorze. Wektor `x` zawiera wartości pierwszej cechy, wektor `y` zawiera wartości drugiej cechy, a `labels` zawiera wartości klasy decyzyjnej.

In [ ]:
from helpers import plot_classification, get_separable
x,y,labels = get_separable()
plot_classification(x,y,labels)

Mamy dane pochodzące z dwóch klas, a nasze zadanie polega na implementacji klasyfikatora SVM czyli indukcji (w naszym dwuwymiarowym przypadku) linii któa będzie separowała obie klasy. 
$$a \cdot x+b \cdot y +c = 0$$
Klasyfikator będzie przypisywał obiekt do jednej z klas gdy będzie się on znajdował po prawej stronie tej linii, jeśli zaś obiekt będzie po lewej jej stronie to przypiszemy obiekt do drugiej z klas. Znajdź taką linię dla powyższych danych, gdzie `x` i `y` to współrzędne punktów, a `labels` przyjmuje wartości $1$ i $-1$ wskazując na klasę obserwacji.

In [ ]:
def construct_svm_opt_problem(xs, ys, labels):
    a, b, c = Variable(), Variable(), Variable()
    objective = Minimize(a**2 + b**2 + c**2)
    indicies_one = np.where(labels==1)
    indicies_minus_one = np.where(labels==-1)
    constraints = [a*xs[indicies_one]+b*ys[indicies_one] + c >= 1, a*xs[indicies_minus_one]+b*ys[indicies_minus_one] + c <= -1]
    prob = Problem(objective, constraints)

    return prob

prob = construct_svm_opt_problem(x, y, labels)
prob.solve()
prob.status

Jeśli dobrze wytrenowałeś klasyfikator to powinno się udać go narysować.

In [ ]:
a, b, c = prob.variables()
plot_classification(x,y,labels, a = a.value, b=b.value, c=c.value)

Przejdźmy na trudniejszy zbiór, który nie jest liniowo separowalny. 

**UWAGA** Dane są losowane - istnieje mała szansa, że akurat wylosujesz zbiór separowalny. W takiej sytuacji powtórz wywołanie.

In [ ]:
from helpers import plot_classification, get_non_separable

x, y, labels = get_non_separable()
plot_classification(x, y, labels)

Rozszerz swoją implementację klasyfikatora SVM tak aby prawidłowo działał również ze zbiorami nieliniowo separowalnymi. Za stałą $C$ możesz przyjąć 1.

In [ ]:
def construct_softsvm_opt_problem(xs, ys, labels, C = 1):
    a, b, c, Eta = Variable(), Variable(), Variable(), Variable(len(labels))
    objective = Minimize(a**2 + b**2 + c**2 + C * sum(Eta**2))
    indicies_one = np.where(labels==1)
    indicies_minus_one = np.where(labels==-1)
    constraints = [a*xs[indicies_one]+b*ys[indicies_one] + c >= 1 - Eta[indicies_one], a*xs[indicies_minus_one]+b*ys[indicies_minus_one] + c <= -1 + Eta[indicies_minus_one]]
    prob = Problem(objective, constraints)

    return prob
    
prob = construct_softsvm_opt_problem(x, y, labels)
prob.solve()
prob.status

Narysuj nauczony klasyfikator.

In [ ]:
a, b, c, Eta = prob.variables()
plot_classification(x,y,labels, a = a.value, b = b.value, c = c.value)

## Zadanie 2. - Eksploracja liniowego SVM (za 20%)

Wczytaj dane liniowo sepratowalne gdzie `X` to macierz cech, a `y` zwiera wartości klasy decyzyjnej.

In [ ]:
from helpers import get_separable
X, y = get_separable(True)

Wytrenuj model maszyn wektorów podpierających używając obiektu SVC (Support Vector Classifier) z biblioteki `sklearn`. Konstruktor tego obiektu ma parametr `kernel` który domyślnie jest ustawiony na jądro RBF, początkowo będziemy chcieli używać wersji klasyfikatora bez jądra czyli inaczej z jądrem liniowym - ustaw ten parametr na wartość `"linear"`.

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(X, y)

Kolejnym krokiem będzie narysowanie granicy decyzji. Funkcję możesz implementować przyrostowo, punkt po punkcie.
1. Niech funkcja narysuje dane uczące (X,y) zaznaczając kolorem klasy decyzyjne.
2. Zaznacz na wykresie granicę decyzji. Wytrenowany SVM (obiekt SVC) ma odpowiednie wartości nauczonych współczynników we właściwości `svm.coef_` a także wyraz wolny w `svm.intercept_`<br>
    *Wskazówka*: Zwróć uwagę, że linię separującą o wzorze
    $w_0*x_0 + w_1*x_1 + b = 0$ można przekształcić do postaci:
    $$x_1= -\frac{w_0}{w_1}  x_0 - \frac{b}{w_1} $$
3. Zaznacz na wykresie margines decyzji (np. poprzez narysowanie dwóch linii równoległych do granicy decyzji w odpowiedniej odległości). Przypomnij sobie, że z naszych wyprowadzeń wynika że wartość marginesu to $\frac{1}{||w||}$
4. Zakreśl specjalnym symbolem lub podświetl elementy zbioru uczącego będącymi wektorami podpierającymi. Ich współrzędne możesz odnaleźć we właściwości `svm.support_vectors_`

In [ ]:
def plot_svm(svm, X, y):
    """ Input: Funkcja na wejście otrzymuje wytrenowany klasyfikator SVM (obiekt SVC) 
        oraz zbiór danych (X - cechy , y - klasa)
        Output: Funkcja rysuje dane oraz zaznacza granicę decyzji
    """    
    # TWÓJ KOD TUTAJ
    plt.scatter(X[:, 0], X[:, 1], c = y)
    x = np.arange(X[:, 0].min(), X[:, 0].max(), 0.02)
    w0 = svm.coef_[0][0]
    w1 = svm.coef_[0][1]
    b = svm.intercept_[0]
    y_x = -x*(w0/w1) - (b/w1)
    margin = 1 / np.linalg.norm(svm.coef_[0])
    plt.plot(x, y_x)
    plt.plot(x, y_x-margin, 'g--')
    plt.plot(x, y_x+margin, 'g--')
    plt.scatter(svm.support_vectors_[:, 0], svm.support_vectors_[:, 1], marker='+')
    print(svm)
    
plot_svm(model, X, y)

Zwróć uwagę na granicę decyzji, narysowany margines oraz podświetlone obiekty będące wektorami podpierającymi.

Wczytaj nowy zbiór danych

In [ ]:
dane = np.load('data_outlier.npz')
X = dane['X']
y = dane['y'].reshape(-1)

oraz powtórz poprzednią procedurę tj. wytrenuj klasyfikator oraz narysuj wykres.

In [ ]:
model = SVC(kernel='linear')
model.fit(X, y)
plot_svm(model, X, y)

W `sklearn` klasyfikator SVM ma domyślną wartość parametru $C = 1.0$. Zbiór jest jednak liniowo separowalny - sprawdź jak wygląda granica decyzji z innymi wartościami $C$ a w szczególności z wartością `C = float("inf")`. Wartość $C$ jest parametrem konstruktora obiektu SVC.

In [ ]:
model = SVC(kernel='linear', C=float('inf'))
model.fit(X, y)
plot_svm(model, X, y)

Wypróbuj różne wartości parametru $C$ na zbiorze nieseparowalnym liniowo. 

In [ ]:
from helpers import get_non_separable
X, y = get_non_separable(True)

In [ ]:
model = SVC(kernel='linear', C=0.001, probability=True)
model.fit(X, y)
plot_svm(model, X, y)

**Polecenia**
1. Jak zmienia się wyindukowana granica decyzji dla bardzo dużych ($\infty$) i bardzo małych (np. 0.01) wartości $C$? Jak zmienia się trafność klasyfikatora na zbiorze uczącym? Jak sądzisz, która granica decyzja będzie najlepsza na zbiorze testowym?
2. Jak zmienia się liczba wektorów podpierających wraz ze zmianą $C$? Liczba wektorów podpierających jest miarą złożoności hipotezy - jakie wartości $C$ powodują bardziej złożone hipotezy? Odnieś te rozważania do przetargu wariancja - obciążenie.
3. Czy klasyfikator SVM jest klasyfikatorem który może zwrócić prawdopodobieństwo przynależności do klas? Czy jest to klasyfikator generatywny czy dyskryminacyjny?

Odpowiedz na powyższe pytania:

1. Dla bardzo dużych C wartości zmiennych osłabiających są jak najmniejsze przez co algorytm preferuje wąskie marginesy nienaruszające prawidłowej klasyfikacji. Dla bardzo małych wartości C większość ograniczeń może zostać wyeliminowana przez co wagi modelu mogą być jak najmniejsze i margines jak największy. Na zbiorze uczącym trafność klasyfikacji będzie największa dla dużych wartości C, jednak może to powodować przesadne dostosowywanie do przypadków odstających i słabszy wynik na zbiorze testowym.
2. Liczba wektorów podpierających wzrasta kiedy wartość C maleje. Większe wartości C dają mniej obciążony model który posiada więcej wariancji i jest bardziej podatny na dostosowanie się do przypadków odstających.
3. Tak może zwrócić prawdopodobieństo przynależności do klasy np. biorąc po uwagę odległość od marginesu. Jest to klasyfikator dyskryminacyjny, ponieważ nie uczy się rozkładu danych, a jedynie tego jak je rozdzielać.

## Zadanie 3. - Trik jądrowy (za 20%)
Klasyfikator SVM, który jest liniowy jest dość mocno ograniczony. Nie będzie potrafił m.in. prawidłowo zaklasyfikować przedstawionego zbioru.

In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=100, noise=0.15, random_state=2024)

model = SVC(kernel="linear")
model.fit(X, y)
plot_svm(model, X, y)

Problem ten można rozwiązać podobnie jak w przypadku regresji logistycznej: poprzez dodawanie dodatkowych cech. Na zajęciach z regresji dodawaliśmy cechy wielomianowe: spróbuj tego tricku również teraz wykorzystując obiekt `Pipeline` oraz `PolynomialFeatures` z cechami stopnia 3. 

**Uwaga**: do rysowania wykresu trzeba teraz użyć funkcji `plot_classifier`.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from helpers import plot_classifier

pipeline = Pipeline([
    ('cubic features', PolynomialFeatures(degree=3, include_bias=False)),
    ('svc', SVC(kernel='linear'))
])

#call the pipeline
pipeline.fit(X, y)
plot_classifier(X, y, pipeline)

Tak jak dyskutowaliśmy na laboratoriach (i wykładzie) tego typu operacja dla zbiorów które mają nawet niezbyt dużo cech może wygenerować bardzo dużo cech. Dla przykładu rozważając zbiór z tysiącem cech i generację cech wielomianowych tylko *drugiego* rzędu (czyli de facto najniższego nieliniowego) spowoduje dodanie prawie pół miliona cech! Nie mniej jednak kernel (jądro) takiej operacji jest wyrażone wzorem:
$(x^Ty + 1 )^2$ i można je obliczyć w czasie liniowym od liczby oryginalnych cech (tutaj 1000)!

W obiekcie SVC możemy wykorzystać jądro wielomianowe specyfikując `kernel="poly"` oraz ew. podając dodatkowe parametry `degree` oraz `coef0`. Co oznaczają te parametry? Jądro wielomianowe jest wyrażone wzorem:
$$K(x,y) = (x^\mathsf{T} y + c)^{d}$$
gdzie $d$ to właśnie `degree`, a stała $c$ to `coef0`. Zmienna $d$ ma bardzo prostą interpretację: jest to stopień wielomianu. Stała $c$ ma natomiast szczególne znaczenie: jeśli jest ona równa $c=0$ to kernel generuje wszystkie jednomiany rzędu $d$ (jądro wielomianowe homogeniczne), jeżeli $c=1$ to kernel generuje wszystkie jednomiany rzędu co najwyżej $d$. Dodatkowo stała $c$ może być też strojona (ale w praktyce raczej rzadko się to robi)  ponieważ reguluje ona przetarg między wpływem cech wyższego rzędu a wpływem cech rzędu niższego. Stała $c$ nie może być ujemna. 

Wykorzystaj jądro wielomianowe do klasyfikacji badanego zbioru. (Pamiętaj o wyelimiowaniu dodawania cech wielomianowych!)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from helpers import plot_classifier

model = SVC(kernel='poly', coef0=1)
model.fit(X, y)
plot_classifier(X, y, model)

SVM z jądrami może sobie teraz poradzić także z problemem XOR. Wygenerujmy takie dane:

In [ ]:
X = np.random.randn(200, 2)
y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)

i wytrenuj klasyfikator. Spróbuj pozmieniać wartość kontrolującą złożoność hipotezy $C$ (nie jest to $c$ z funkcji jądra!).

In [ ]:
model = SVC(kernel='poly', coef0=1)
model.fit(X, y)
plot_classifier(X, y, model)

Posteruj parametrami `degree` i `C` dla problemu XOR oraz wcześniejszego problemu klasyfikacji (dwa księżyce).

Prawdopodobnie najczęściej stosowaną funkcją jądrową jest jądro RBF (*Radial basis function kernel*) wyrażone wzorem:
$${\displaystyle K(\mathbf {x} ,\mathbf {x'} )=\exp(-\gamma \|\mathbf {x} -\mathbf {x'} \|^{2})}$$
który inaczej zapisany ${\displaystyle K(\mathbf {x} ,\mathbf {x'} )=\exp \left(-{\frac {\|\mathbf {x} -\mathbf {x'} \|^{2}}{2\sigma ^{2}}}\right)}$ przypomina wzór na rozkład normalny (przy $\gamma^{-1} = 2\sigma ^{2}$). Jego działanie możemy sobie wyobrazić jako stosowanie rozkładu normalnego, który jest wycentrowany na rozważanym przykładzie $\mathbf{x}$ do mierzenia podobieństwa z innymi przykładami. Przypisuje on najwyższe "prawdopodobieństwo" (wartość funkcji jądrowej) do naszej obserwacji $\mathbf{x}$ a im dalej od niej tym wartość spada. Jeśli więc inny przykład $\mathbf{x'}$ leży blisko $\mathbf{x}$ w przestrzeni cech to jądro odpowie wysoką wartością, im dalej tym odpowiedź będzie niższa. Z kształtu rozkładu normalnego wiemy też, ze po przekroczeniu pewnej odległości ($3\sigma$) jądro będzie odpowiadało zawsze (prawie) zerem. Szybkość opadania funkcji jądra reguluje właśnie parametr $\sigma$: im jest on wyższy, tym wariancja (szerokość) rozkładu jest wyższa, więc jądro wolniej spada. Innymi słowy przykłady mają "większy zakres oddziałowywawania" na inne przykłady. W kontekście jądra RBF przyjęło się sterować parametrem $\gamma$, który jak już podawałem wynika z podstawienia:
$$\gamma = \frac{1}{2\sigma ^{2}}$$
czyli im wyższe $\gamma$ (niższe $\sigma$) tym mniejszy zakres oddziałowywania, funkcja szybciej spada z odległością od przykładu i na odwrót. 

Niezwykłą ciekawostką dot. tej funkcji jądrowej jest to, że gdybyśmy zastanowili się jak wygląda przestrzeń cech które powinniśmy wygenerować aby dostać taki wynik bez użycia triku jądrowego to okazałoby się, że jest ich... nieskończenie wiele!

Jak pewnie się domyślasz, zastąpienie w konstruktorze `SVC` wyboru jądra na `kernel="rbf"` oraz podanie parametru `gamma` umożliwi ci przetestowanie SVM z jądrem RBF. 

In [ ]:
X = np.random.randn(200, 2)
y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)

def plot_rbf(gamma):
    model = SVC(kernel='rbf', gamma=gamma)
    model.fit(X, y)
    plot_classifier(X, y, model)
    
plot_rbf(2)

In [ ]:
for gamma in [0.01, 0.1, 1, 10]:
    plot_rbf(gamma)
    plt.show()

In [ ]:
X, y = make_moons(n_samples=100, noise=0.05)

for gamma in [0.5, 0.7, 1, 1.5, 1.7, 2]:
    model = SVC(kernel='rbf', gamma=gamma)
    model.fit(X, y)
    print(gamma)
    plot_classifier(X, y, model)
    plt.show()

In [ ]:
X = np.random.rand(2000,2)*20
y = np.sin(X[:,0])+np.sin(X[:,1])>0

for gamma in [0.5, 0.7, 1, 1.5, 1.7, 2]:
    model = SVC(kernel='rbf', gamma=gamma)
    model.fit(X, y)
    print(gamma)
    plot_classifier(X, y, model)
    plt.show()

**Polecenia**: 
1. Przetesuj kilka wartości `gamma`. Rozważ przynajmniej wartości 0.01, 0.1, 1, 10.
2. Jakie wartości `gamma` powodują przeuczenie, a jakie niedouczenie?
3. Wczytaj poprzedni problem z dwoma księżycami i spróbuj dobrać `gamma`.
4. Wygeneruj zbiór z szachownicą
    ```
    X = np.random.rand(2000,2)*20
    y = np.sin(X[:,0])+np.sin(X[:,1])>0
    ```
    i spróbuj wybrać odpowiednio sparametryzowaną funkcję jądrową.

5. Czy klasyfikator SVM jest odporny na zmianę skali? Innymi słowy czy powinno się normalizować cechy przed jego użyciem? Dlaczego?
6. Jak myślisz czy dla każdego problemu (lub ew. prawie każdego) jest możliwe takie dobranie jądra np. wielomianowego aby zbiór był liniowo separowalny? Tj. byłoby w tej przestrzeni możliwe nauczenie klasyfikatora SVM z $C=\infty$? (Możesz spróbować, ale nie musisz)
7. Klasyfikator SVM jest binarny. Jak mógłbyś go użyć do klasyfikacji danych wieloklasowych?

Odpowiedź na punkty 2., 5. i 7. wpisz poniżej (krótkie konkretne odowiedzi wystarczą), dla reszty pozostaw kod w komórkach i ich wyniki jako dowód Twoich eksperymentów.

2 Niedouczenie dla: 0.01 i 0.1. Przeuczenie dla 10

5 Nie jest odporny, nieznormalizowane skale mogą negatywnie wpłynąć na dopór najlepszego marginesu

7 Wytrenować wiele klasyfikatorów każdy do odseparowania jednej klasy od wszystkich pozostałych, następnie przy klasyfikacji wybrać ten który jest najbardziej pewny (predict_proba), albo podzielić zbiór na wiele zbiorów gdzie w każdym mamy tylko dwie klasy, następnie wytrenować svm na wszystkich tych zbiorach, przy klasyfikacji wybieramy klasę która jest wybierana najczęściej.

## Zadanie 4. - Wybór parametrów (za 20%)

Wygeneruj dane z szachownicą (takie same jak w zadanich wyżej)

In [ ]:
X = np.random.rand(2000,2)*20
y = np.sin(X[:,0])+np.sin(X[:,1])>0
plt.scatter(X[:,0], X[:,1], c=y)

Spróbuj wybrać optymalne parametry dla klasyfikatora SVM z jądrem RBF (weź pod uwagę parametr `gamma` jądra i parametr `C`). Przykładową techniką poszukiwania dobrych parametrów jest zdefiniowanie tzw. siatki parametrów i sprawdzenie wszystkich możliwości. W pierwszym kroku musimy ustalić siatkę czyli np. dla parametru `gamma` będziemy testować wartości 0.1, 1, 10, a dla parametru `C` wartości 1 i 10. Poszukiwanie gridowe dla takiej siatki uruchomi proces uczenia 6 razy, zewaluuje trafność i zwróci najlepsze parametry. Ewaluacja trafności może się odbywać np. walidacją krzyżową, a w `sklearn` jest to zaimpementowane w obiekcie `GridSearchCV`.

Pierwszym argumentem konstruktora obiektu `GridSearchCV` jest obiekt klasyfikatora, a kolejne ważne parametry to ` param_grid= słownik z siatką`, `scoring='accuracy'` (aby była ewaluowana trafność). Dodatkowo można ustawić `verbose=2` jeśli chcesz, żeby pojawiły się na konsoli dodatkowe informacji o uczeniu kolejnych klasyfikatorów oraz parametrem `cv=` kontrulującym na ile cześci powinien zostać podzielony zbiór uczący w trakcie walidacji krzyżowej. Np.
```
GridSearchCV(KLASYFIKATOR, param_grid= PARAMETRY scoring='accuracy', verbose=2, cv=3)
```
Siatkę `PARAMETRY` definiujemy poprzez podanie słownika, którego kluczami są nazwy parametrów klasyfikatora, a wartościami są listy wartości do sprawdzenia np. `{'gamma':[0.1,1,10], 'C':[1,10]}`. `GridSearchCV` uruchamiamy standardowo czyli porzez funkcję `fit()`. 

Znajdź dobre ustawienia klasyfikatora SVM z jądrem RBF dla problemu z szachownicą.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

model = SVC(kernel='rbf')
params = dict(gamma=[0.1,1,10], C=[1,10])
grid = GridSearchCV(model, param_grid=params, scoring='accuracy', cv=3, n_jobs=4)
grid.fit(X, y)

Przyjrzyj się wynikowi - właściwość `GRID.best_params_`

In [ ]:
grid.best_params_ 

Oraz szczegółom przeszukiwania - właściwość `GRID.cv_results_`

In [ ]:
grid.cv_results_

Wytrenuj klasyfikator z najlepszymi znalezionymi parametrami i zwizualizuj go `plot_classifier`.

In [ ]:
model = SVC(kernel='rbf', **grid.best_params_)
model.fit(X, y)
plot_classifier(X, y, model)

In [ ]:
X, y = make_moons(n_samples=100, noise=0.2)

param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
  {'C': [1, 10, 100, 1000], 'degree': [2, 3, 4], 'kernel': ['poly']},
 ]

model = SVC()
grid = GridSearchCV(model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=4)
grid.fit(X, y)

In [ ]:
grid.best_params_ 

In [ ]:
grid.cv_results_

**Polecenia**
1. Typ funkcji jądrowej może być jednym z parametrów które stroimy. Jednak różne funkcje jądrowe mają różne parametry - jak to wyspecyfikować? Zobacz fragment [dokumentacji sekcja 3.2.1 ](https://scikit-learn.org/stable/modules/grid_search.html#exhaustive-grid-search), który pokazuje jak to zrobić. Rozszerz twoje przeszukiwanie o jądra liniowe i wielomianowe.
2. Spróbuj strojenia na innym wybranym zbiorze.
3. Czy wynik uzyskany przez GridSearchCV jest dobrą estymacją trafności na zbiorze testowym?

Odpowiedź na punkt 3. poniżej.

3. Tak, ponieważ używa walidacji krzyżowej

## Zadanie 5. - SVM dla dużych danych (za 20%)

In [ ]:
from helpers import get_wine
X, y = get_wine()

Metoda maszyn wektorów podpierających ma wiele zalet. Po pierwsze tworzy modele z dość małą liczbą wektorów podpierających które determinują wielkość nauczonego modelu. Dodatkowo faza predykcji jest bardzo szybka. Ponieważ uczenie jest zależne tylko od kilku, automatycznie wybieranych, wektorów podpierających, a resztę zbioru danych można by usunąć bez zmiany wyindukowanego klasyfikatora to wynik SVM zwykle zależy od małego podzbioru zbioru uczącego. SVM dobrze sobie radzą nawet w sytuacji gdy liczba przykładów jest dalece mniejsza niż liczba cech! (Przypomnij sobie co się wtedy dzieje z regresją liniową). Dodatkowo, pomimo tego że SVM jest klasyfikatorem liniowym to zastosowanie triku jądrowego pozwala na indukcje płaszczyzny separującej w rozszerzonej przestrzeni. Ponadto trik jądrowy pozwala na tworzenie własnych jąder dostosowanych do dziedziny problemu (tak jak w kNN mogliśmy projektować funkcje odległości) co spowodowało  szerokie ich wykorzystanie w tekstach czy obrazach (istnieją gotowe funkcje jądrowe nawet do przetwarzania grafów).

Jeśli chodzi o najważniejsze wady tego klasyfikatora to pierwszą dużą wadą jest konieczność strojenia parametrów funkcji jądrowych oraz parametru $C$. Jednakże w świetle powszechnego korzystania z sieci neuronowych, które wymagają strojenia znacznie większej liczby parametrów (architektura, różne dodatkowe triki) nie wiem czy nadal powinno się to uważać za  wadę nie do przeskoczenia. Druga wada SVM jest jednak znaczenie gorsza: koszt obliczeniowy uczenia. Rozwiązywanie problemu programowania kwadratowego definiowanego przez SVM jest rzędu sześciennego od liczby przykładów $\mathcal{O}[n^3]$ czy dla efektywniejszych implementacji $\mathcal{O}(\max(n,d) \min (n,d)^2)$  - nadal jest to bardzo kosztowne obliczeniowo.

Jednakże na ten drugi problem możemy rozwiązać za pomocą algorytmów w rodzaju stochastycznego spadku wzdłuż gradientu! Funkcja celu która jest optymalizowana przez SVM nazywamy funkcją zwiasową (*hinge loss*) i jest ona ekwiwalentna z prymalną postacią problemu. Jednak to właśnie jego postać dualna umożliwiała zastosowanie triku jądrowego. Aby więc zastosować SGD trzeba by rzeczywiście przeskalować zbiór uczący do przestrzeni cech definiowanej przez jądro (ona może mieć nawet $\infty$ liczbę wymiarów!). Na szczęście tak jak SGD używa przybliżonej estymacji gradientu tak samo możemy użyć *przybliżonej* transformacji jądrowej. Jedną z takich technik jest aproksymacja Nyströma, która wychodzi z założenia że macierz jądrowa jest rzędu $m$ dalece mniejszego od $n$. Zakłada się więc, że mając tylko wiedzę o $m$ wierszach macierzy jądrowej można zrekonstruować ją całą. W praktyce nie jest to prawda, ale po prostu mamy nadzieję, że $n-m$ najmniejszych wartości własnych jest na tyle małe że i tak uzyskujemy dobrą aproksymację (patrz: Materiały dla chętnych).

Wytrenuj na wczytanych danych SVM poprzez algorytm SGD - obiekt `SGDClassifier`. Pamiętaj o wyspecyfikowaniu zawiasowej funkcji straty `loss='hinge'`. Dodatkowo, aby wykonać trik jądrowy przetransformuj dane wejściowe aprokymacją Nyströma (obiekt `Nystroem`). Parametry tego obiektu to `kernel` (ustawmy RBF) oraz `gamma` (dla RBF), a także ` n_components` czyli ile wektorów używanych jest do przybliżenia jądra (domyślnie 100 i do naszych celów jest to wystarczające).

In [ ]:
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)
nyst = Nystroem(kernel='rbf', n_components=100)
X_tr_nyst = nyst.fit_transform(X_tr)
X_te_nyst = nyst.fit_transform(X_te) # tu powinno być samo transform chyba?

Porównaj czas uczenia na tych samych danych modelu `SVC` oraz modelu przybliżonego (`Nystroem`+`SGD`). Czas wykonania komórki kodu możesz przetestować poprzez dodanie do niej na początku znacznika 
`%%timeit`.

In [ ]:
%%timeit
model = SVC(kernel='rbf')
model.fit(X_tr, y_tr)

In [ ]:
%%timeit
model = SGDClassifier(loss='hinge')
model.fit(X_tr_nyst, y_tr)

In [ ]:
repeat_factor = 5
X_repeat = np.repeat(X, repeat_factor).reshape([X.shape[0] * repeat_factor, X.shape[1]])
y_repeat = np.repeat(y, repeat_factor).reshape(np.array(y.shape)*repeat_factor)
X_tr, X_te, y_tr, y_te = train_test_split(X_repeat, y_repeat, test_size=0.2)
nyst = Nystroem(kernel='rbf', n_components=100)
X_tr_nyst = nyst.fit_transform(X_tr)
X_te_nyst = nyst.transform(X_te) # tutaj bez fit

In [ ]:
# %%timeit
model_svc = SVC(kernel='rbf')
model_svc.fit(X_tr, y_tr)
model_svc.score(X_te, y_te)

In [ ]:
# %%timeit
model_sgd = SGDClassifier(loss='hinge')
model_sgd.fit(X_tr_nyst, y_tr)
model_sgd.score(X_te_nyst, y_te)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

repeat_factor = 5
X_repeat = np.repeat(X, repeat_factor).reshape([X.shape[0] * repeat_factor, X.shape[1]])
y_repeat = np.repeat(y, repeat_factor).reshape(np.array(y.shape)*repeat_factor)
X_tr, X_te, y_tr, y_te = train_test_split(X_repeat, y_repeat, test_size=0.2)
scaler = MinMaxScaler()
X_tr = scaler.fit_transform(X_tr)
X_te = scaler.transform(X_te)
nyst = Nystroem(kernel='rbf', n_components=100)
X_tr_nyst = nyst.fit_transform(X_tr)
X_te_nyst = nyst.transform(X_te)

In [ ]:
# %%timeit
model_svc = SVC(kernel='rbf')
model_svc.fit(X_tr, y_tr)
model_svc.score(X_te, y_te)

In [ ]:
# %%timeit
model_sgd = SGDClassifier(loss='hinge')
model_sgd.fit(X_tr_nyst, y_tr)
model_sgd.score(X_te_nyst, y_te)

In [ ]:
import time
repeat_factor = 5
X_repeat = np.repeat(X, repeat_factor).reshape([X.shape[0] * repeat_factor, X.shape[1]])
y_repeat = np.repeat(y, repeat_factor).reshape(np.array(y.shape)*repeat_factor)
X_tr, X_te, y_tr, y_te = train_test_split(X_repeat, y_repeat, test_size=0.2)

for n_components in [10, 50, 100, 200, 300, 1000]:
    nyst = Nystroem(kernel='rbf', n_components=n_components)
    X_tr_nyst = nyst.fit_transform(X_tr)
    X_te_nyst = nyst.transform(X_te)

    start = time.time()
    model_sgd.fit(X_tr_nyst, y_tr)
    end = time.time()
    print(f'n_components: {n_components}, score: {model_sgd.score(X_te_nyst, y_te)}, time: {end - start:.02f}s')

**Polecenia**
1. Zwiększ rozmiar zbioru pięciokrotnie (kopiując przykłady). Jak teraz wygląda czas uczenia klasyfikatora  obiema metodami?
2. Porównaj trafności obydwu metod na zbiorze uczącym i testowym.
3. Spróbuj przeskalować dane uczące przed użyciem obydwu metod np. metodą Min-Max
   ```
   from sklearn.preprocessing import MinMaxScaler
   scaler = MinMaxScaler()
   ```
   Jak zmienił się czas uczenia? Jak zmieniła się trafność modelu?

4. Przetestuj różne wartości `n_components` transformacji przybliżonej. Spróbuj zaobserwować zależności pomiędzy trafnością modelu a modelem referencyjnym (bez przybliżenia). Możesz też spróbować porównać czasy uczenia dla kliku wielkości `n_components`.

Zostaw kod w komórkach i ich wyniki na dowód Twoich eksperymentów.

1. SVC: 17.2s, SGDClassifier: 117ms
2. SVC: 0.80, SGDClassifier: 0.81
3. SVC: [14s, 0.80] (czas się skrócił, trafność ta sama), SGDClassifier: [141ms, 0.75] (czas się wydłużył, a trafność zmniejszyła)